In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image
import os

# Device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
# Parameters
latent_dim = 100 # synthetic neural data size
image_size = 28*28
batch_size = 64
num_epochs = 200
save_dir = './generated_images'
real_images = './real_images'
os.makedirs(save_dir, exist_ok=True)
os.makedirs(real_images, exist_ok=True)


In [ ]:
# Create synthetic neural data
def generate_synthetic_neural_data(batch_size, dim=latent_dim):
    return torch.randn(batch_size, dim).to(device)

In [12]:
# Generator
class Generator(nn.Module):
    def __init__(self, input_dim, img_shape):
        super(Generator, self).__init__()
        self.img_shape = img_shape
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(1024, int(torch.prod(torch.tensor(img_shape)))), 
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(z.size(0), *self.img_shape)
    
# Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(img_shape))), 512),

            nn.Linear(512, 256), 
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(256, 1), 
            nn.Sigmoid()
        )
    def forward(self, img):
        flat = img.view(img.size(0), -1)
        return self.model(flat)

In [20]:
# Initialize models
img_shape = (1, 28, 28)
generator = Generator(latent_dim, img_shape).to(device)
discriminator = Discriminator(img_shape).to(device)

# Loss and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    # Generate data
    real_imgs = torch.randn(batch_size, *img_shape).to(device)
    z = generate_synthetic_neural_data(batch_size)

    # Ground truths
    valid = torch.ones(batch_size, 1).to(device)
    fake = torch.zeros(batch_size, 1).to(device)

    # --- Train Generator ---
    optimizer_G.zero_grad()
    gen_imgs = generator(z)
    g_loss = adversarial_loss(discriminator(gen_imgs), valid)
    g_loss.backward()
    optimizer_G.step()

    # --- Train Discriminator ---
    optimizer_D.zero_grad()
    real_loss = adversarial_loss(discriminator(real_imgs), valid)
    fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
    d_loss = (real_loss + fake_loss) / 2
    d_loss.backward()
    optimizer_D.step()

    # Save progress
    if epoch % 10 == 0:
        print(f"[Epoch {epoch}/{num_epochs}] D loss: {d_loss.item():.4f} | G loss: {g_loss.item():.4f}")
        save_image(gen_imgs.data[:25], f"{save_dir}/epoch_{epoch}.png", nrow=5, normalize=True)
        save_image(real_imgs.data[:25], f"{real_images}/epoch_{epoch}_real.png", nrow=4, normalize=True)


[Epoch 0/200] D loss: 0.6819 | G loss: 0.6966
[Epoch 10/200] D loss: 0.6398 | G loss: 0.7657
[Epoch 20/200] D loss: 0.6428 | G loss: 0.7527
[Epoch 30/200] D loss: 0.6313 | G loss: 0.7791
[Epoch 40/200] D loss: 0.5294 | G loss: 1.0332
[Epoch 50/200] D loss: 0.5052 | G loss: 1.1105
[Epoch 60/200] D loss: 0.4944 | G loss: 1.3233
[Epoch 70/200] D loss: 0.4271 | G loss: 1.7806
[Epoch 80/200] D loss: 0.4421 | G loss: 1.7702
[Epoch 90/200] D loss: 0.4793 | G loss: 1.8011
[Epoch 100/200] D loss: 0.4943 | G loss: 2.0740
[Epoch 110/200] D loss: 0.4861 | G loss: 2.0274
[Epoch 120/200] D loss: 0.4780 | G loss: 1.4115
[Epoch 130/200] D loss: 0.5187 | G loss: 1.1615
[Epoch 140/200] D loss: 0.5607 | G loss: 1.0825
[Epoch 150/200] D loss: 0.5510 | G loss: 1.1172
[Epoch 160/200] D loss: 0.5360 | G loss: 1.0352
[Epoch 170/200] D loss: 0.5248 | G loss: 1.1519
[Epoch 180/200] D loss: 0.4882 | G loss: 1.3849
[Epoch 190/200] D loss: 0.4931 | G loss: 1.2581
